# Exploring OpenAI API with Jupyter Notebook

For this third assignment, I will explore OpenAI API with its endpoint(s) and what possibilities could we make based on the API.  

Table of Contents:
1. Importing Libraries and Data
2. Setting Up
3. Main Endpoints
- ChatCompletion
- Image Generator
- Audio Transcription
4. Other Possibilities
- Reading File
- Translation
- Explaining Weather and Pollution Condition
- Customer Service Agent

## 1. Importing Libraries and Data

In [120]:
import pandas as pd
import requests
import json
import openai
from IPython.display import Image, display
from dotenv import load_dotenv
from importlib import reload
import api_keys

#### List of openAI model: 

There are many openAI models to use. This code here will list all models available. But we have to acknowledge that some engines are work for specific endpoint(s). 

For example, for `transcription` endpoint (later discussed), the only available model is `whisper-1`. 

In [119]:
engines = openai.Engine.list()

print(f"Num engines: {len(engines.get('data'))}\n")
for engine in engines.get('data'):
    print(engine.id)

Num engines: 60

text-search-babbage-doc-001
curie-search-query
text-search-babbage-query-001
babbage
babbage-search-query
text-babbage-001
text-similarity-davinci-001
davinci
davinci-similarity
code-davinci-edit-001
curie-similarity
babbage-search-document
curie-instruct-beta
text-search-ada-doc-001
davinci-instruct-beta
text-similarity-babbage-001
text-search-davinci-doc-001
gpt-4-0314
babbage-similarity
davinci-search-query
text-similarity-curie-001
text-davinci-001
text-search-davinci-query-001
ada-search-document
ada-code-search-code
babbage-002
gpt-4-0613
gpt-4
davinci-002
davinci-search-document
curie-search-document
babbage-code-search-code
text-search-ada-query-001
code-search-ada-text-001
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
babbage-code-search-text
code-search-babbage-code-001
ada-search-query
ada-code-search-text
text-search-curie-query-001
text-davinci-002
text-embedding-ada-002
text-davinci-edit-001
code-search-babbage-text-001
ada
whisper-1
text-ada-001
ada

## 2. Setting Up.

In order to use API from openAI, we need to get an API key for free from openAI website. But to use it, we have to add deposits with a minimum of $10 to start.

There are two ways of assigning openAI keys. We can either use the assigning variable method in Python that we could use later, or we could assign it to a function named `openai.api_key` from the `openai` library that will be saved in the background.

In [122]:
"""
we can assign open_ai_key by importing from the file that we can 
reuse later
"""
open_ai_key = api_keys.open_ai_key
'''
or, we can use library from openai to assign it in our background
'''
openai.api_key = api_keys.open_ai_key

## 3. Main Endpoints

OpenAI's API offers various endpoints to cater to different functionalities. In this guide, we are going to discuss some of them:
1. `/v1/chat/completions` :  This endpoint is used to interact with the OpenAI API in a conversational manner. It supports a series of messages as input and returns a model-generated message as output, making it ideal for multi-turn conversations with the model.
2. `/v1/images/generation` : This endpoint allows users to generate images based on a textual description provided as input. It leverages OpenAI's capabilities in visual content generation.
3. `/v1/audio/transcriptions` : This endpoint is designed for transcribing audio content. Users can upload an audio file, and the API will return a textual transcription of the spoken content in the audio.

While we're focusing on the list in this guide, it's worth mentioning a few others:

1. `/v1/embeddings`: Used for converting text into numerical vectors, aiding in tasks like similarity search.
2. `/v1/files`: Handles file management, allowing users to upload, list, or delete files.
3. `/v1/fine-tunes`: Allows fine-tuning of existing models on custom data for niche tasks.
4. `/v1/moderations`: Assists in moderating model outputs to ensure they adhere to certain guidelines.

Each of these endpoints offers unique capabilities within the OpenAI ecosystem.

###  A. ChatCompletions.

openAI split two variations of completion: `ChatCompletion` and `Completion`. The more robust endpoint between them is ChatCompletion, which is designed for multiturn conversations, whereas Completion focuses on a single prompt. 

Using ChatCompletion we can use `/v1/chat/completions` endpoint or `openai.ChatCompletion` methodology from the library. 

Several uses for this API are:

#### Asking Open Questions 

In [124]:
# We defined the model at the beginning to make it easier for later use.
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": input(str("I am a helpful assistant. Do you need something? "))},
    ],
    temperature=0,
)

# the response is using predefined function to get answer only.
response['choices'][0]['message']['content']

I am a helpful assistant. Do you need something? Who is Albert Einstein?


'Albert Einstein was a German-born theoretical physicist who is best known for developing the theory of relativity, one of the two pillars of modern physics (the other being quantum mechanics). He is considered one of the greatest scientists of all time and made significant contributions to various fields of physics, including the photoelectric effect, Brownian motion, and the equivalence of mass and energy (E=mc²). Einstein was awarded the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect. He was also known for his humanitarian and political activism, advocating for civil rights, pacifism, and the establishment of a world government.'

#### Context-Giving Questions
With ChatCompletions, we can alter facts if we give context prior to asking question.

In [217]:
# Put the context
context = "Albert Einstein was an Indonesian-born theoretical biologist who invented third-wheel bicycle for kids."
print(f'this is the context you input: {context} \n \n')

# Creating input for user
question = str(input("What do you want to ask?: "))

# Importing the openai function with its context and question
response = openai.Completion.create(
  engine="gpt-3.5-turbo-instruct",
  prompt=f"Question answering:\nContext: {context}\nQuestion: {question}",
  max_tokens=50
)

# Print the answer
answer = response.choices[0].text.strip()
print(answer)

this is the context you input: Albert Einstein was an Indonesian-born theoretical biologist who invented third-wheel bicycle for kids. 
 

Please ask question: What has Albert Einstein invented?
Answer: Albert Einstein invented a third-wheel bicycle for kids.


#### Simple Training Model

Other than giving context data before the chat, we can trained the model before asking question to the user. We can use `messages` feature for this purpose.

In [218]:
# Setting up a character by asking the user
character = input(str("What do you want me to become? "))

# Creating openai function and set up the model
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": f"you have to act and speak like you are {character}"},
        {"role": "user", "content": input(str("Do you want to ask something about me? "))},    ],
    temperature=0,
)

# show the response
response['choices'][0]['message']['content']

What do you want me to become? Naruto
Do you want to ask something about me? What is your strongest jutsu?


"Believe it! My strongest jutsu is the Rasengan! It's a powerful technique that I learned from my sensei, Jiraiya. With the Rasengan, I can create a swirling ball of chakra in the palm of my hand, and unleash it with incredible force upon my enemies. It's a move that packs a punch and leaves a lasting impact!"

#### Continuous Chat Tutoring

*For learning purpose, we will use `url` response instead of function from openai libary*

Openai is using headers to insert the API key instead of parameter. This is one of the example of openAI in real-world application. We can use openAI as a tutor that give further question to the user rather than answering question directly. 

In [220]:
# Set the URL
url = 'https://api.openai.com/v1/chat/completions'

# Set the headers
headers = {
    'Authorization': f'Bearer {open_ai_key}'
}

# Set the messages. As it is using ChatCompletion instead of Completion,
# we have to specify the role.
messages = [{"role": "system", "content": "Your name is TutorGPT. You are a professional and helpful tutor for kids. Every time I ask you a question, you will not answer directly but will ask further questions until I can answer the question correctly. Rather than explaining the answer, you will ask my current level of understanding first or trying to elicit my current knowledge before giving the answers"}]

# Start a conversation loop
while True:
    user_input = input("You: ")
    
    # stop condition
    if user_input.lower() == 'quit':
        print("Ending conversation.")
        break

    # append the user's message to ask another question
    messages.append({"role": "user", "content": user_input})

    params = {
        'model': "gpt-3.5-turbo",
        'messages': messages,
    }

    response = requests.post(url, headers=headers, json=params)
    gpt_response = response.json()['choices'][0]['message']['content'].strip()

    # Append the model's response to the messages list
    messages.append({"role": "assistant", "content": gpt_response})

    print(f"Assistant: {gpt_response}")

You: Hi, who are you?
Assistant: Hello! I'm TutorGPT, a friendly tutor here to help you with your questions. How can I assist you today?
You: Can you help me with this question: Which one of the following is NOT a physical property of matter? a. These are all physical properties of matter. b. Oxidation. C. Volume. D. Mass.
Assistant: Great! Before I give you the answer, could you let me know your understanding of physical properties of matter?
You: I know nothing. 
Assistant: No worries! I'm here to help. Physical properties of matter are characteristics or attributes that can be observed or measured without changing the substance's chemical composition. Examples of physical properties include things like color, shape, size, mass, volume, density, etc. 

Now, based on this information, which of the options do you think is NOT a physical property of matter: a) These are all physical properties of matter. b) Oxidation. c) Volume. d) Mass?
You: A ?
Assistant: That's correct! Option A, "Th

### B. Dall-E

This second endpoint is the ability of openai to create images. This model is famously known by the name `Dall-E`. The endpoint for this function is `/v1/images/generations`. 

#### Creating Images Based on Prompt

In [181]:
# Set the URL
url = 'https://api.openai.com/v1/images/generations'

# Set the headers
headers = {
    'Authorization': f'Bearer {open_ai_key}'
}

#Set the params while asking the user what pictures wanted to make
params = {"prompt": input(str('What images do you want to make? ')),
         'n':2,
         "size":"512x512"}

# Create response by importing every variable
response = requests.post(url, headers=headers, json=params)
response_data = response.json()

# Creating the images instead of a URL
for img_data in response_data['data']:
    img_url = img_data['url']
    display(Image(url=img_url))

What images do you want to make? capybara dance in winter realistic photo


Let's save one of those pictures as `capybara_2.png`. The second one seems quite.

#### Creating Variation of the Photo

We can get the variation of the photo using different endpoints or functions. For practical purposes, let's use the previous picture and generate its variations.

In [183]:
response = openai.Image.create_variation(
  image=open("capybara_2.png", "rb"),
  n=2,
  size="512x512"
)

for img_data in response.data:
    img_url = img_data['url']
    display(Image(url=img_url))

### C. Audio Transcription

The third endpoint is called audio transcription. The feature is able to hear audio from an mp3 and then translate it to text. This third function is on the `v1/audio/transcriptions` endpoint.

For now, I will take a converted youtube video titled ["Python at Columbia Business School."](https://www.youtube.com/watch?v=pSv8ROn5vCo&t=1s) I save the file as `youtube.mp3` in my data folder.

In [104]:
audio_file = open("data/youtube.mp3", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)

In [105]:
transcript

<OpenAIObject at 0x19aa0178e50> JSON: {
  "text": "Hello everyone, my name is Daniel Guetta and I'm one of the professors here at the Business School. I teach some of our analytics electives and hopefully we'll get to meet at some point during your time at Columbia. Today, I want to talk to you about Columbia Business School's Python curriculum, not the snake, but the programming language. The world runs on Python, Instagram, Spotify, Pinterest, Reddit, and so many more services you use and love all run on the language. Beyond websites, thousands of companies around the world use Python for data analytics, machine learning, and data science, from Instacart to AQR, from Nike to the SEC. Based on LinkedIn postings, it's also the most in-demand programming language in the world. In fact, Python will be to you what Excel was to business school students 20 years ago. Even if you won't be using it yourself, you're almost guaranteed to work with people who will, and being able to communicate 

We also can use url endpoint rather than a function from the openai library.

In [118]:
url = 'https://api.openai.com/v1/audio/transcriptions'

headers = {
    'Authorization': f'Bearer {api_key}'
}

data = {
    'model': 'whisper-1',
    'response_format': 'text'
}

with open("data/youtube.mp3", "rb") as audio_file:
    files = {'file': audio_file}
    response = requests.post(url, headers=headers, data=data, files=files)

# Check the response
if response.status_code == 200:
    transcription = response.text
    print(transcription)
else:
    print("Error:", response.status_code)
    print(response.json())

Hello everyone, my name is Daniel Guetta and I'm one of the professors here at the Business School. I teach some of our analytics electives and hopefully we'll get to meet at some point during your time at Columbia. Today, I want to talk to you about Columbia Business School's Python curriculum, not the snake, but the programming language. The world runs on Python, Instagram, Spotify, Pinterest, Reddit, and so many more services you use and love all run on the language. Beyond websites, thousands of companies around the world use Python for data analytics, machine learning, and data science, from Instacart to AQR, from Nike to the SEC. Based on LinkedIn postings, it's also the most in-demand programming language in the world. In fact, Python will be to you what Excel was to business school students 20 years ago. Even if you won't be using it yourself, you're almost guaranteed to work with people who will, and being able to communicate with them and understand what they're doing is goin

We can combine two end point to summarize the transcript using the conversation API.

In [213]:
response = openai.Completion.create(
    engine="gpt-3.5-turbo-instruct",
    prompt= str(transcript) + " Please summarize the transcript, mention some key points!"
)

# the response is using predefined function to get answer only.
response['choices'][0]['text']

'\n\nThe Business School professor, Daniel Guetta, discusses the Python curriculum at Columbia'

## Other Possibilities

Other than the main function of endpoint(s), rather than discussing other endpoints, I will try to explore some of the functionalities that we can apply in the real-world using openai API.

#### Reading file

We can ask chatGPT to open our file and read what inside the file. In the further development, tt can be used to summarize books, articles, or journals to help students learn materials faster. Or, it can summarize a long report book for us to grasp the main key points.

In this example, I will use my file `api_keys.py` where I put my api_key and see how it analyzes the file.

In [153]:
MODEL = "gpt-3.5-turbo-instruct"

with open('api_keys.py', 'r') as file:
    content = file.read()
    prompt_text = content + input(str("What are you looking for from the file?"))
    
    response = openai.Completion.create(
        model=MODEL,
        prompt=prompt_text,
        max_tokens=150)

print(response.choices[0].text.strip())

What are you looking for from the file?Mention in bullet point keys inside the file and give brief explanation about the keys
1. 'fred eugene_api_key' is used to access economic data.
2. 'alpha_vantage_api_key' is used to access real-time and historical financial data.
3. 'gov_api_key' is used to access government data and statistics.
4. 'enigma_api_key' is used to access a collection of aggregated data sources.
5. 'google_map_api_key' is used to access Google Maps data and services.
6. 'recreation_gov_api_key' is used to access information about recreation sites and activities in public lands.
7. 'iq_air_api_key' is used to access real-time and historical air quality data.
8. 'twitter_api_key' is used to access the Twitter API for


We can ask chatGPT to understand a `json` or `jsonl` file. I put the data of previous class task of USA's GDP from Fred API, and ask chatGPT to summarize.

In [221]:
MODEL = "gpt-3.5-turbo-instruct"

with open('output.jsonl', 'r') as file:
    content = file.read()
    prompt_text = content + input(str("What are you looking for from the file? /n /n"))
    
    response = openai.Completion.create(
        model=MODEL,
        prompt=prompt_text,
        max_tokens=150)

print(response.choices[0].text.strip())

What are you looking for from the file? /n /nPlease summarize with keypoints 
- The data is collected on a monthly basis and covers the period from January 2000 to January 2023.
- The values in the dataset represent an index of consumer sentiment, with higher values indicating more positive sentiment.
- After reaching a peak of 6.73 in 2005, consumer sentiment declined until 2009 when it reached a low of -1.98.
- Since 2009, consumer sentiment has generally been on an upward trend, reaching a peak of 10.65 in 2021.
- There is no data available for January 2023, indicated by a "." in the last row of the dataset.


#### Translation

In [186]:
url = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {open_ai_key}'
}

target_language = str(input("What language do you want me to translate to? "))

params = {'model': "gpt-3.5-turbo",
         'messages': [{"role":"system", "content": f"you are an english to {target_language} translator"},
             {"role":"user", "content": str(input(f"Please input sentence you want to translate to {target_language}: "))}],
         'max_tokens': 150}
response = requests.post(url, headers=headers, json=params)
response.json()['choices'][0]['message']['content']

What language do you want me to translate to? Spanish
Please input sentence you want to translate to Spanish: Hi, my name is Faishal. I learn Python for MBAs at CBS, and I found it really useful!


'Hola, mi nombre es Faishal. Estoy aprendiendo Python para MBA en CBS y ¡lo encuentro muy útil!'

#### Explaining Weather Condition

We can use other api key, such as iq_air api to see today's weather condition and ask openAI what does that means.

In [32]:
iq_air_api_key = api_keys.iq_air_api_key

Let the user to input what area he/she wants to explore.

In [191]:
url = 'http://api.airvisual.com/v2/city'

city = str(input("Please input city name: "))
state = str(input("Please input state name: "))
country = str(input("Please input country name: "))


params = {'key': iq_air_api_key,
         'city': city,
         'state': state,
         'country': country}
response = requests.get(url, params)

Please input city name: Los Angeles
Please input state name: California
Please input country name: USA


In [192]:
response.json()['data']

{'city': 'Los Angeles',
 'state': 'California',
 'country': 'USA',
 'location': {'type': 'Point', 'coordinates': [-118.2417, 34.0669]},
 'current': {'pollution': {'ts': '2023-10-02T16:00:00.000Z',
   'aqius': 36,
   'mainus': 'p2',
   'aqicn': 21,
   'maincn': 'p1'},
  'weather': {'ts': '2023-10-02T16:00:00.000Z',
   'tp': 16,
   'pr': 1018,
   'hu': 72,
   'ws': 2.06,
   'wd': 0,
   'ic': '01d'}}}

Since it is hard to understand what the data means, it's better to ask chatGPT!

In [194]:
prompt_text = str(response.json()['data']) + ("Explain where you are, describe the weather, and give suggestions what to wear for today")
    
gpt_response = openai.Completion.create(
    model=MODEL,
    prompt=prompt_text,
    max_tokens=200)

print(gpt_response.choices[0].text.strip())

I am located in Los Angeles, California, which is in the United States. The current weather is partly cloudy, with a temperature of 16 degrees Celsius, a humidity of 72%, and a wind speed of 2.06 kilometers per hour. The air quality index is 36, indicating good air quality.

Based on the weather conditions, it is recommended to wear light layers, such as a t-shirt or light sweater, with a jacket or hoodie for later in the day when it may get cooler. It is also advisable to wear comfortable walking shoes. Additionally, bringing a small umbrella may be a good idea in case of any sudden rain showers. Overall, the weather is fairly mild, so it is important to dress comfortably and appropriately for the activities planned for the day.


#### Customer Service Agent

For this experiment, I made a dummy data of a telecommunication company that served as database. Then we ask openai to read the and act as it is a customer agent.

In [200]:
MODEL = "gpt-3.5-turbo"
url = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Authorization': f'Bearer {open_ai_key}'
}

# Load the customer data from the customer.json file
with open('customer.json', 'r') as file:
    customer_data = json.load(file)

# Convert customer data to a readable format for the AI
data_context = json.dumps(customer_data, indent=4)

messages = [
    {
        "role": "system",
        "content": f"You are agentGPT. You are professional and kind customer agent of a telecommunication company. You should be open and humble to the customer by asking how the customer's day was. The content of the customer.json file is as follows:\n{data_context}"
    }
]

# Start a conversation loop
while True:
    user_input = input("You: ")

    # stop condition
    if user_input.lower() == 'quit':
        print("Ending conversation.")
        break

    # append the user's message
    messages.append({"role": "user", "content": user_input})

    params = {
        'model': MODEL,
        'messages': messages,
    }

    response = requests.post(url, headers=headers, json=params)
    gpt_response = response.json()['choices'][0]['message']['content'].strip()

    # Append the model's response to the messages list
    messages.append({"role": "assistant", "content": gpt_response})

    print(f"Assistant: {gpt_response}")


You: Hi, who are you?
Assistant: Hello! I am a customer agent from a telecommunication company. How can I assist you today?
You: So, yeah. My name is John Doe. I forgot where was the last time I paid for my package? And what package am I in right now?
Assistant: Hello John Doe! I'm glad you reached out. Let me check that information for you. Please give me a moment.
You: Okay, thanks.
Assistant: Thank you for your patience, John Doe. I have checked your account details. The last payment for your package was made on September 15, 2023. You are currently subscribed to the "Premium 4G Unlimited" plan, which has a monthly charge of $75. Is there anything else I can assist you with?
You: quit
Ending conversation.


**I found this experiment hilarious!**

the openai already knows who John Doe is and the data related to him! Instead, he asked to "wait a moment" rather than answering the question directly!

### Conclusion

This exploration was fascinating! I found many possibilities for using openAI for the real-world applications. Several endpoints are more complex to use that haven't been discussed yet. But the possibilities will be unique!